In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMClassifier
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler
SEED = 0

In [2]:
types_dict = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float16',
    'prior_question_had_explanation': 'boolean'
}
train_df = pd.read_csv(
    'data/train.csv',
    usecols=types_dict.keys(),
    dtype=types_dict,
    index_col=0
)
question_df = pd.read_csv('data/questions.csv')

In [3]:
features_df = train_df.iloc[:int(9 /10 * len(train_df))]
train_df = train_df.iloc[int(9 /10 * len(train_df)):]
print(features_df.shape, train_df.shape)

(91107298, 6) (10123034, 6)


In [4]:
train_questions_only_df = features_df[features_df['answered_correctly']!=-1]

In [5]:
new_column = list(question_df.columns)
new_column[0] = 'content_id'
question_df.columns = new_column
question_df = question_df[['content_id', 'part']]

In [6]:
train_questions_only_df = train_questions_only_df.merge(question_df, how='left', on='content_id')

In [7]:
train_questions_only_df.head()

,timestamp,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part
0,0,115,5692,1,NaN,<NA>,5
1,56943,115,5716,1,36992.0,False,5
2,118363,115,128,1,55008.0,False,1
3,131167,115,7860,1,19008.0,False,1
4,137965,115,7922,1,11000.0,False,1


In [8]:
grouped_by_user_df = train_questions_only_df.groupby('user_id')
user_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew']}).copy()
user_answers_df.columns = ['mean_user_accuracy', 'questions_answered', 'std_user_accuracy', 'median_user_accuracy', 'skew_user_accuracy']
user_question_part_df = grouped_by_user_df.agg({'part': ['mean', 'count', 'std']}).copy()
user_question_part_df.columns = ['mean_user_question_part', 'count_question_part', 'std_question_part']

grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew'] }).copy()
content_answers_df.columns = ['mean_accuracy', 'question_asked', 'std_accuracy', 'median_accuracy', 'skew_accuracy']

grouped_by_part_df = train_questions_only_df.groupby('part')
part_answers_df = grouped_by_part_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew'] }).copy()
part_answers_df.columns = ['mean_part_accuracy', 'question_part_answered', 'std_part_accuracy', 'median_part_accuracy', 'skew_part_accuracy']

In [9]:
user_answers_df.head()

,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy
user_id,,,,,
115,0.695652,46,0.465215,1.0,-0.879359
124,0.233333,30,0.430183,0.0,1.328338
2746,0.578947,19,0.507257,1.0,-0.347892
5382,0.672000,125,0.471374,1.0,-0.741648
8623,0.642202,109,0.481566,1.0,-0.601619


In [10]:
user_question_part_df.head()

,mean_user_question_part,count_question_part,std_question_part
user_id,,,
115,1.521739,46,1.149669
124,4.400000,30,1.734041
2746,2.315789,19,0.945905
5382,3.816000,125,1.608365
8623,3.366972,109,2.084602


In [11]:
content_answers_df.head()

,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy
content_id,,,,,
0,0.908595,6236,0.288207,1.0,-2.836339
1,0.891682,6684,0.310805,1.0,-2.521185
2,0.554656,40499,0.497010,1.0,-0.219949
3,0.779348,20734,0.414696,1.0,-1.347371
4,0.613226,28549,0.487020,1.0,-0.465009


In [12]:
part_answers_df.head()

,mean_part_accuracy,question_part_answered,std_part_accuracy,median_part_accuracy,skew_part_accuracy
part,,,,,
1,0.745481,6726438,0.435591,1,-1.127118
2,0.708740,16884487,0.454343,1,-0.918864
3,0.701916,7763910,0.457416,1,-0.882856
4,0.631078,7270010,0.482513,1,-0.543316
5,0.610045,36815816,0.487740,1,-0.451246


In [13]:
del features_df
del grouped_by_user_df
del grouped_by_content_df
del grouped_by_part_df

gc.collect()

177

In [14]:
features = [
    'mean_user_accuracy', 
    'questions_answered',
    'std_user_accuracy', 
    'median_user_accuracy',
    'skew_user_accuracy',
    'mean_accuracy', 
    'question_asked',
    'std_accuracy', 
    'median_accuracy',
    'skew_accuracy',
    'mean_part_accuracy',
    'question_part_answered',
    'std_part_accuracy',
    'median_part_accuracy',
    'skew_part_accuracy',
    'mean_user_question_part',
    'count_question_part',
    'std_question_part',
    'prior_question_elapsed_time', 
    'prior_question_had_explanation'
]
target = 'answered_correctly'
train_df = train_df[train_df[target] != -1]

In [15]:
train_df = train_df.merge(question_df, how='left', on='content_id')
train_df = train_df.merge(user_answers_df, how='left', on='user_id')
train_df = train_df.merge(user_question_part_df, how='left', on='user_id')
train_df = train_df.merge(content_answers_df, how='left', on='content_id')
train_df = train_df.merge(part_answers_df, how='left', on='part')

In [16]:
train_df.head()

,timestamp,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,part,mean_user_accuracy,questions_answered,std_user_accuracy,...,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy,mean_part_accuracy,question_part_answered,std_part_accuracy,median_part_accuracy,skew_part_accuracy
0,8231964660,1933715875,11259,0,13000.0,True,5,0.779843,5219.0,0.414392,...,0.532146,1291.0,0.499159,1.0,-0.128999,0.610045,36815816,0.48774,1,-0.451246
1,8232002976,1933715875,4957,1,44000.0,True,5,0.779843,5219.0,0.414392,...,0.584772,2548.0,0.492858,1.0,-0.344273,0.610045,36815816,0.48774,1,-0.451246
2,8232096407,1933715875,5113,1,22000.0,True,5,0.779843,5219.0,0.414392,...,0.603571,1960.0,0.489280,1.0,-0.423795,0.610045,36815816,0.48774,1,-0.451246
3,8232119872,1933715875,4699,1,inf,True,5,0.779843,5219.0,0.414392,...,0.694888,2504.0,0.460547,1.0,-0.847011,0.610045,36815816,0.48774,1,-0.451246
4,8232142930,1933715875,11430,1,9000.0,True,5,0.779843,5219.0,0.414392,...,0.765869,1922.0,0.423565,1.0,-1.256695,0.610045,36815816,0.48774,1,-0.451246


In [17]:
train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].fillna(value = False).astype(bool)
train_df = train_df.fillna(value = -1)
train_df = train_df[features + [target]]
train_df = train_df.replace([np.inf, -np.inf], np.nan)
train_df = train_df.fillna(0)

In [18]:
train_df.head()

,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy,...,question_part_answered,std_part_accuracy,median_part_accuracy,skew_part_accuracy,mean_user_question_part,count_question_part,std_question_part,prior_question_elapsed_time,prior_question_had_explanation,answered_correctly
0,0.779843,5219.0,0.414392,1.0,-1.351136,0.532146,1291.0,0.499159,1.0,-0.128999,...,36815816,0.48774,1,-0.451246,4.290094,5219.0,1.675174,13000.0,True,0
1,0.779843,5219.0,0.414392,1.0,-1.351136,0.584772,2548.0,0.492858,1.0,-0.344273,...,36815816,0.48774,1,-0.451246,4.290094,5219.0,1.675174,44000.0,True,1
2,0.779843,5219.0,0.414392,1.0,-1.351136,0.603571,1960.0,0.489280,1.0,-0.423795,...,36815816,0.48774,1,-0.451246,4.290094,5219.0,1.675174,22000.0,True,1
3,0.779843,5219.0,0.414392,1.0,-1.351136,0.694888,2504.0,0.460547,1.0,-0.847011,...,36815816,0.48774,1,-0.451246,4.290094,5219.0,1.675174,0.0,True,1
4,0.779843,5219.0,0.414392,1.0,-1.351136,0.765869,1922.0,0.423565,1.0,-1.256695,...,36815816,0.48774,1,-0.451246,4.290094,5219.0,1.675174,9000.0,True,1


In [19]:
train_df, test_df = train_test_split(train_df, random_state=SEED, test_size=0.2)

In [20]:
sampler = TPESampler(seed=666)

def create_model(trial):
    num_leaves = trial.suggest_int("num_leaves", 2, 31)
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 8)
    min_child_samples = trial.suggest_int('min_child_samples', 100, 1200)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0001, 0.99)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 5, 90)
    bagging_fraction = trial.suggest_uniform('bagging_fraction', 0.0001, 1.0)
    feature_fraction = trial.suggest_uniform('feature_fraction', 0.0001, 1.0)
    model = LGBMClassifier(
        num_leaves=num_leaves,
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_child_samples=min_child_samples, 
        min_data_in_leaf=min_data_in_leaf,
        learning_rate=learning_rate,
        feature_fraction=feature_fraction,
        random_state=SEED
    )
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(train_df[features], train_df[target])
    score = roc_auc_score(test_df[target].values, model.predict_proba(test_df[features])[:,1])
    return score

# uncomment to use optuna
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=70)

params = study.best_params
params['random_state'] = SEED
# params = {
#     bagging_fraction=0.15974843888522386,
#     feature_fraction=0.7192650047521456,
#     learning_rate=0.3152716923851695, 
#     max_depth=7,
#     min_child_samples=695, 
#     min_data_in_leaf=31, 
#     n_estimators=244,
#     num_leaves=13, 
#     random_state=0
# }
model = LGBMClassifier(**params)
model.fit(train_df[features], train_df[target])

ignored. Current value: feature_fraction=0.9222006560140852
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=926 will be ignored. Current value: min_data_in_leaf=11
[I 2020-10-25 16:47:56,104] Trial 40 finished with value: 0.7218473766323035 and parameters: {'num_leaves': 17, 'n_estimators': 192, 'max_depth': 8, 'min_child_samples': 926, 'learning_rate': 0.7354475480981951, 'min_data_in_leaf': 11, 'bagging_fraction': 0.18097265067756166, 'feature_fraction': 0.9222006560140852}. Best is trial 1 with value: 0.7221725183566224.
[LightGBM] [Warning] feature_fraction is set=0.6755495477788396, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6755495477788396
[LightGBM] [Warning] min_data_in_leaf is set=71, min_child_samples=1107 will be ignored. Current value: min_data_in_leaf=71
[I 2020-10-25 16:48:24,083] Trial 41 finished with value: 0.7220979457686002 and parameters: {'num_leaves': 19, 'n_estimators': 167, 'max_depth': 8, 'min_child_samples': 110

LGBMClassifier(bagging_fraction=0.15974843888522386,
               feature_fraction=0.7192650047521456,
               learning_rate=0.3152716923851695, max_depth=7,
               min_child_samples=695, min_data_in_leaf=31, n_estimators=244,
               num_leaves=13, random_state=0)

In [21]:
print('LGB score: ', roc_auc_score(test_df[target].values, model.predict_proba(test_df[features])[:,1]))

LGB score:  0.7222313798427497


In [ ]:
'''
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(question_df, how='left', on='content_id')
    test_df = test_df.merge(user_answers_df, how = 'left', on = 'user_id')
    test_df = test_df.merge(user_question_part_df, how='left', on='user_id')
    test_df = test_df.merge(content_answers_df, how = 'left', on = 'content_id')
    test_df = test_df.merge(part_answers_df, how='left', on='part')
    test_df = test_df.merge(user_lecture_part_df, how='left', on='user_id')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value = False).astype(bool)
    test_df.fillna(value = -1, inplace = True)

    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:,1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
'''